# <b> Buy and Hold </b>

Example showing the backtesting of a Buy and Hold strategy.

When the strategy starts, it will buy 50% of AAPL and 50% of GOOG.
As soon as the backtest completes, it will return a dataframe 'result' containing the results, which can be used to visualize the equity's evolution and other metrics, like when buy or sell orders are placed.

## <b> Import Libraries </b>

In [2]:
# Import this library
from nbacktest import Backtest, Strategy

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## <b> Create Strategy </b>

In [3]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = int((self.balance/2)/self.prices["AAPL"])
        quantity2 = int((self.balance/2)/self.prices["GOOG"])

        order1 = self.buy("AAPL", quantity1)
        order2 = self.buy("GOOG", quantity2)

        trade1 = self.create_trade(orders=[order1, order2], notes="Buy and Hold Trade")

        print("Bought %s AAPL and %s GOOG" % (quantity1, quantity2))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass


## <b> Download OHLC data from Yahoo Finance </b>

In [4]:
universe = ["AAPL", "GOOG"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2018-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(1259, 10)


Price            Close                   High                    Low  \
Ticker            AAPL       GOOG        AAPL       GOOG        AAPL   
Date                                                                   
2022-12-23  129.900299  89.141418  130.451974  89.429260  127.713291   
2022-12-27  128.097504  87.275421  129.456999  88.833733  126.806975   
2022-12-28  124.166786  85.816360  129.082622  87.861023  123.999315   
2022-12-29  127.683731  88.287819  128.540796  88.699730  125.831674   
2022-12-30  127.998947  88.069473  128.018654  88.168727  125.536110   

Price                        Open               Volume            
Ticker           GOOG        AAPL       GOOG      AAPL      GOOG  
Date                                                              
2022-12-23  86.967727  128.974267  86.967727  63814900  17815000  
2022-12-27  86.883365  129.427446  88.645145  69007800  15470900  
2022-12-28  85.727034  127.742834  86.848619  85438400  17879600  
2022-12-29  86.342411  126.087805  86.382114  75703700  18280700  
2022-12-30  86.382124  126.501549  86.714629  77034200  19190300

## <b> Run Backtest </b>

In [5]:
bt = Backtest(data=df_ohlc,
              universe=universe,
              strategy_class=TestStrategy,
              price_column="Close",
              cash=100_000
             )

result = bt.run()

result.head()

Strategy started!
Bought 1239 AAPL and 946 GOOG
Strategy finished!


,BALANCE,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED_TOTAL,EQUITY
ITERATION,,,,
2018-01-02,16.901165,99983.098835,0,100000.000000
2018-01-03,16.901165,100795.039742,0,100811.940907
2018-01-04,16.901165,101211.191517,0,101228.092682
2018-01-05,16.901165,102526.006077,0,102542.907242
2018-01-08,16.901165,102558.522003,0,102575.423168


## <b> Backtest Statistics </b>

In [6]:
print("---------------------------")
print("Final balance: %.2f" % bt.strategy.balance)
print("Final equity: %.2f" % bt.strategy.equity)
print("---------------------------")

---------------------------
Final balance: 16.90
Final equity: 241921.32
---------------------------


In [7]:
bt.get_trade_statistics()

{'n_won': 1,
 'n_lost': 0,
 'n_total': 1,
 'win_rate': 1.0,
 'mean_profit': 141921.31838607788,
 'median_profit': 141921.31838607788,
 'mean_profit_win': 141921.31838607788,
 'mean_profit_loss': nan,
 'median_profit_win': 141921.31838607788,
 'median_profit_loss': nan,
 'std_profit': nan,
 'mad_profit': 0.0,
 'skewness': nan,
 'kurtosis': nan,
 'mean_median_gap': 0.0,
 'mean_profit_over_vol': nan,
 'median_profit_over_vol': nan}

In [8]:
bt.get_equity_statistics_dollar()

{'total_profit': 141921.31838607788,
 'mean_profit': 112.81503846270101,
 'median_profit': 256.2001552581787,
 'std_profit': 4002.75432308696,
 'mad_profit': 2424.4090544971386,
 'skewness': -0.19350273676472046,
 'kurtosis': 3.0569005307430164,
 'mean_median_gap': -143.3851167954777,
 'mean_profit_over_vol': 0.028184352412540033,
 'median_profit_over_vol': 0.06400596553739897,
 'max_drawdown': -122839.07723236084}

In [9]:
bt.get_equity_statistics_return()

{'total_return': 1.4192131838607787,
 'mean_return': 0.0008838017809516843,
 'median_return': 0.0016635019416982866,
 'std_return': 0.01902778652310216,
 'mad_return': 0.014074704582391169,
 'skewness': -0.15457311684246133,
 'kurtosis': 4.0711563409885825,
 'mean_median_gap': -0.0007797001607466023,
 'mean_return_over_vol': 0.04644795546127429,
 'median_return_over_vol': 0.08742487938250637,
 'max_drawdown': -0.3432401003550529}

In [10]:
bt.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE,AGE
0,6057aae0-a8c0-401b-97f2-eae145118251,OPEN,Buy and Hold Trade,141921.318386,2018-01-02,None,None,"{'AAPL': {'quantity': 1239, 'value': 158590.69...",241904.417221,{},0,None,None,None,1823 days


In [11]:
bt.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL
0,2e90995b-0aec-494b-ad54-b2b03cca65ac,AAPL,BUY,FILLED,6057aae0-a8c0-401b-97f2-eae145118251,2018-01-02,1239,40.341892,0.0,49983.604488,49983.604488,2018-01-02,1239,40.341892,0.0,-49983.604488,-49983.604488
1,ff83480b-7882-4312-8b44-25bbd43c0015,GOOG,BUY,FILLED,6057aae0-a8c0-401b-97f2-eae145118251,2018-01-02,946,52.853588,0.0,49999.494347,49999.494347,2018-01-02,946,52.853588,0.0,-49999.494347,-49999.494347


In [12]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.69551086426)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}

In [17]:
bt.strategy.positions_unfilled

{}

In [14]:
bt.strategy.positions_filled

{'AAPL': {'quantity': 1239, 'value': np.float64(158590.69551086426)},
 'GOOG': {'quantity': 946, 'value': np.float64(83313.72171020508)}}